<a href="https://colab.research.google.com/github/rajanaids-hub/Reinforcement_Learning_Lab/blob/main/Simplified_Policy_Iteration_Simply_Exp7V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML, display

# =====================================================================
# 🎓 STUDENT EXPERIMENT SETTINGS 🎓
# Try changing these values to see how the agent's behavior changes!
# =====================================================================
GRID_SIZE = 4           # Try 5 or 6 (Creates an NxN grid)
DISCOUNT_FACTOR = 1.0   # Try 0.9, 0.5, or 0.1 (How much does it care about the future?)
STEP_REWARD = -1.0      # Try -10 or -0.1 (The penalty for taking a step)
# =====================================================================

# 1. Define the Environment
class GridworldEnv:
    """
    A simple grid where the agent must reach the top-left or bottom-right corner.
    """
    def __init__(self, size=GRID_SIZE):
        self.size = size
        self.nS = size * size  # Total states
        self.nA = 4            # 4 Actions: 0=UP, 1=RIGHT, 2=DOWN, 3=LEFT
        self.P = {}            # P[state][action] = (probability, next_state, reward, is_done)

        # Build the grid rules
        for state in range(self.nS):
            self.P[state] = {a: [] for a in range(self.nA)}
            is_goal = (state == 0 or state == self.nS - 1)

            if is_goal:
                # If at the goal, stay there and get 0 reward
                for a in range(self.nA):
                    self.P[state][a] = [(1.0, state, 0.0, True)]
            else:
                row, col = divmod(state, self.size)
                for a in range(self.nA):
                    next_row, next_col = row, col

                    if a == 0: next_row = max(row - 1, 0)               # UP
                    elif a == 1: next_col = min(col + 1, self.size - 1) # RIGHT
                    elif a == 2: next_row = min(row + 1, self.size - 1) # DOWN
                    elif a == 3: next_col = max(col - 1, 0)             # LEFT

                    next_state = (next_row * self.size) + next_col
                    self.P[state][a] = [(1.0, next_state, STEP_REWARD, False)]

# 2. Policy Evaluation
def evaluate_policy(policy, env, gamma):
    """ Calculate how good each state is under the current policy. """
    V = np.zeros(env.nS) # Start with value 0 for all states

    while True:
        max_change = 0
        for state in range(env.nS):
            new_value = 0

            # Look at all possible actions from this state
            for action, action_prob in enumerate(policy[state]):
                for prob, next_state, reward, done in env.P[state][action]:
                    # 🧠 The Bellman Equation!
                    new_value += action_prob * prob * (reward + gamma * V[next_state])

            # Track the biggest change to know when to stop
            max_change = max(max_change, np.abs(new_value - V[state]))
            V[state] = new_value

        # Stop if the values have barely changed
        if max_change < 0.0001:
            break

    return V

# 3. Policy Improvement
def improve_policy(env, policy, V, gamma):
    """ Update the policy to pick the best action based on the new values. """
    policy_stable = True

    for state in range(env.nS):
        old_action = np.argmax(policy[state])

        # Check all actions to see which one looks best now
        action_values = np.zeros(env.nA)
        for action in range(env.nA):
            for prob, next_state, reward, done in env.P[state][action]:
                action_values[action] += prob * (reward + gamma * V[next_state])

        best_action = np.argmax(action_values)

        # Did the best action change?
        if old_action != best_action:
            policy_stable = False

        # Update policy: 100% chance to take the best action, 0% for others
        policy[state] = np.eye(env.nA)[best_action]

    return policy, policy_stable

# =====================================================================
# 📉 VISUALIZATION CODE (Students don't need to edit this part)
# =====================================================================
def animate_learning(history, size):
    fig, ax = plt.subplots(figsize=(6, 6))
    arrows = {0: '↑', 1: '→', 2: '↓', 3: '←'}

    def update(frame):
        ax.clear()
        V, policy = history[frame]
        ax.matshow(V.reshape(size, size), cmap='Blues')

        for s in range(len(V)):
            row, col = divmod(s, size)
            ax.text(col, row - 0.2, f"{V[s]:.1f}", va='center', ha='center', color='black')

            if s != 0 and s != len(V) - 1:
                best_act = np.argmax(policy[s])
                ax.text(col, row + 0.2, arrows[best_act], va='center', ha='center', color='red', fontsize=16, weight='bold')
            else:
                ax.text(col, row + 0.2, "GOAL", va='center', ha='center', color='green', fontsize=10, weight='bold')

        ax.set_xticks([]); ax.set_yticks([])
        ax.set_title(f"Iteration: {frame}", pad=20)

    anim = animation.FuncAnimation(fig, update, frames=len(history), interval=1000, repeat=False)
    plt.close(fig)
    display(HTML(anim.to_jshtml()))

# =====================================================================
# 🚀 MAIN EXECUTION
# =====================================================================
if __name__ == "__main__":
    env = GridworldEnv(size=GRID_SIZE)

    # Start with a random policy (25% chance for up, down, left, right)
    policy = np.ones([env.nS, env.nA]) / env.nA
    history = [(np.zeros(env.nS), policy.copy())]

    print("Starting Policy Iteration...")
    iteration = 1

    while True:
        # Step 1: Evaluate
        V = evaluate_policy(policy, env, DISCOUNT_FACTOR)

        # Step 2: Improve
        policy, is_stable = improve_policy(env, policy, V, DISCOUNT_FACTOR)

        history.append((V.copy(), policy.copy()))

        if is_stable:
            print(f"🎉 Found optimal policy in {iteration} iterations!")
            break

        iteration += 1

    # Show the animation
    animate_learning(history, GRID_SIZE)

Starting Policy Iteration...
🎉 Found optimal policy in 3 iterations!
